In [1]:
%load_ext autoreload
%autoreload 2
from models.bilstm_crf import BiLstm_CRF
import torch
import torch.optim as optim
import utils.solver as solver
from models.bilstm_crf import BiLstm_CRF
import utils.data_reader as data_reader



In [ ]:
torch.manual_seed(1)

START_TAG = "<START>"
STOP_TAG = "<END>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
(word2idx, idx2word), (tag2idx, idx2tag) = data_reader.read_word_and_tag(
        './mid/vocab.txt', './data/lab.txt')
seqs_train, tags_train, intents_train = data_reader.read_seqtag_data_with_unali_act(
        './data/train.txt', word2idx, tag2idx)
training_data = list(zip(seqs_train, tags_train))
# print(training_data)
model = BiLstm_CRF(word2idx, tag2idx, EMBEDDING_DIM, HIDDEN_DIM, 1)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = training_data[0][0]
    precheck_tags = training_data[0][1]
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(10):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        targets = torch.tensor(tags, dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.loss(sentence, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = training_data[0][0]
    print(model(precheck_sent))
# We got it!

Reading source data ...
([43, 49, 68, 39, 86, 39, 86, 39, 86, 39, 86, 39, 86, 39, 86], tensor(51.9042))


In [7]:
a = [1,2,3]
b = [5,6,7]
list(zip(a,b))

[(1, 5), (2, 6), (3, 7)]